# Instantiate ICTV OLS Client

In [77]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json

from ictv_ols import ICTVOLSClient
client = ICTVOLSClient('https://www.ebi.ac.uk/ols4/api/v2/ontologies/ictv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get taxa by release

In [78]:

msl_7_viruses = client.get_all_taxa_by_release('MSL7')
print(f"Number of viruses in MSL7: {len(msl_7_viruses)}")

df = pd.DataFrame(msl_7_viruses)[['msl', 'ictv_id', 'label']]
print(df.head(10).to_markdown())



Number of viruses in MSL7: 1221
|    | msl   | ictv_id      | label                      |
|---:|:------|:-------------|:---------------------------|
|  0 | MSL7  | ICTV19710002 | Papovaviridae              |
|  1 | MSL7  | ICTV19710003 | Picornaviridae             |
|  2 | MSL7  | ICTV19710008 | Papillomavirus             |
|  3 | MSL7  | ICTV19710009 | Polyomavirus               |
|  4 | MSL7  | ICTV19710010 | Calicivirus                |
|  5 | MSL7  | ICTV19710011 | Enterovirus                |
|  6 | MSL7  | ICTV19710012 | Rhinovirus                 |
|  7 | MSL7  | ICTV19710013 | Mastadenovirus             |
|  8 | MSL7  | ICTV19710014 | Alfalfa mosaic virus group |
|  9 | MSL7  | ICTV19710015 | Alphavirus                 |


# Get specific taxon by release

In [79]:

picornaviridae = client.get_taxon_by_release('Picornaviridae', 'MSL7')
print(json.dumps(picornaviridae, indent=2))


{
  "msl": "MSL7",
  "ictv_id": "ICTV19710003",
  "label": "Picornaviridae",
  "is_obsolete": true,
  "obsolescence_reason": null,
  "taxonomic_parents": [
    "http://purl.obolibrary.org/obo/NCBITaxon_10239"
  ],
  "replaces": [
    "http://ictv.global/id/MSL6/ICTV19710003"
  ],
  "replaced_by": [
    "http://ictv.global/id/MSL8/ICTV19710003"
  ],
  "current_replacements": [
    "http://ictv.global/id/MSL40/ICTV19710003"
  ]
}


### And get taxonomic parents of a specific taxon in a specific release

In [80]:

parents = client.get_taxonomic_parents('Cardiovirus', 'MSL7')

df = pd.DataFrame(parents)[['ictv_id', 'label']]
print(df.head(10))



        ictv_id           label
0  ICTV19710003  Picornaviridae


# Getting the history of a taxon

### What happened to Bovine adenovirus from MSL6?

In [81]:
taxon = client.get_taxon_by_release('Bovine adenovirus', 'MSL6')
replacements = client.get_current_replacements('Bovine adenovirus', 'MSL6')

print(f"Obsolescence reason: {taxon['obsolescence_reason']}")
print(f"Replacements:")

print(pd.DataFrame(replacements)[['msl', 'ictv_id', 'label']].to_markdown())



Obsolescence reason: SPLIT
Replacements:
|    | msl   | ictv_id      | label                     |
|---:|:------|:-------------|:--------------------------|
|  0 | MSL17 | ICTV19810186 | Bovine adenovirus 4       |
|  1 | MSL17 | ICTV19810187 | Bovine adenovirus 5       |
|  2 | MSL17 | ICTV19810188 | Bovine adenovirus 6       |
|  3 | MSL17 | ICTV19810189 | Bovine adenovirus 7       |
|  4 | MSL17 | ICTV19810190 | Bovine adenovirus 8       |
|  5 | MSL40 | ICTV19810183 | Mastadenovirus bosprimum  |
|  6 | MSL40 | ICTV19810185 | Mastadenovirus bostertium |
|  7 | MSL40 | ICTV19990516 | Mastadenovirus caesari    |
|  8 | MSL40 | ICTV19990521 | Mastadenovirus bovidae    |


### Where did Poliovirus in MSL18 come from?

In [82]:
parents = client.get_historical_parents('Poliovirus', 'MSL18')

print(pd.DataFrame(parents)[['msl', 'ictv_id', 'label', 'obsolescence_reason']].to_markdown())

|    | msl   | ictv_id      | label              | obsolescence_reason   |
|---:|:------|:-------------|:-------------------|:----------------------|
|  0 | MSL17 | ICTV19710068 | Human poliovirus 1 | MERGED                |
|  1 | MSL17 | ICTV19790556 | Human poliovirus 2 | MERGED                |
|  2 | MSL17 | ICTV19790557 | Human poliovirus 3 | MERGED                |


### Where did Alphacytorhabdovirus in MSL40 come from?

In [83]:
parents = client.get_historical_parents('Alphacytorhabdovirus', 'MSL40')

print(pd.DataFrame(parents)[['msl', 'ictv_id', 'label', 'obsolescence_reason']].to_markdown())


|    | msl   | ictv_id      | label           | obsolescence_reason   |
|---:|:------|:-------------|:----------------|:----------------------|
|  0 | MSL39 | ICTV19820133 | Cytorhabdovirus | SPLIT                 |


#### ... and what else was Cytorhabdovirus split into?

In [84]:
replacements = client.get_current_replacements('Cytorhabdovirus', 'MSL39')

print(pd.DataFrame(replacements)[['msl', 'ictv_id', 'label']].to_markdown())

|    | msl   | ictv_id       | label                |
|---:|:------|:--------------|:---------------------|
|  0 | MSL40 | ICTV202419770 | Alphacytorhabdovirus |
|  1 | MSL40 | ICTV202419771 | Betacytorhabdovirus  |
|  2 | MSL40 | ICTV202419772 | Gammacytorhabdovirus |


# What happened to MSL38 Stockinghallvirus?

It was deleted, so there are no current replacements. An empty array is returned.

In [85]:
replacements = client.get_current_replacements('Stockinghallvirus', 'MSL38')

print(replacements)



[]


# Given historical virus names, what are their current replacements?

In [86]:

historical_names = [
    'Severe acute respiratory syndrome coronavirus',
    'Poliovirus type 1',
    'Entomopoxvirus',
    'Poxvirus',
    'Cytorhabdovirus'
]

results = []

for name in historical_names:
    replacements = client.get_current_replacements(name)
    for r in replacements:
        results.append({
            "historical_name": name,
            "current_replacement_label": r['label'],
            "current_replacement_id": r['ictv_id'],
            "current_replacement_msl": r['msl']
        })

print(pd.DataFrame(results).to_markdown())



|    | historical_name                               | current_replacement_label   | current_replacement_id   | current_replacement_msl   |
|---:|:----------------------------------------------|:----------------------------|:-------------------------|:--------------------------|
|  0 | Severe acute respiratory syndrome coronavirus | Betacoronavirus pandemicum  | ICTV20040588             | MSL40                     |
|  1 | Poliovirus type 1                             | Enterovirus coxsackiepol    | ICTV20083305             | MSL40                     |
|  2 | Entomopoxvirus                                | Alphaentomopoxvirus         | ICTV19910168             | MSL40                     |
|  3 | Entomopoxvirus                                | Betaentomopoxvirus          | ICTV19910169             | MSL40                     |
|  4 | Poxvirus                                      | Poxviridae                  | ICTV19740004             | MSL40                     |
|  5 | Cytorhabdovir

# Given historical virus IDs, what are their current replacements?

In [87]:

historical_ids = [
    'ICTV20040588',
    'ICTV19710068',
    'ICTV19740021',
    'ICTV19710040',
    'ICTV19820133'
]

results = []

for id in historical_ids:
    replacements = client.get_current_replacements(id)
    for r in replacements:
        results.append({
            "historical_id": id,
            "current_replacement_label": r['label'],
            "current_replacement_id": r['ictv_id'],
            "current_replacement_msl": r['msl']
        })

print(pd.DataFrame(results).to_markdown())

|    | historical_id   | current_replacement_label   | current_replacement_id   | current_replacement_msl   |
|---:|:----------------|:----------------------------|:-------------------------|:--------------------------|
|  0 | ICTV20040588    | Betacoronavirus pandemicum  | ICTV20040588             | MSL40                     |
|  1 | ICTV19710068    | Enterovirus coxsackiepol    | ICTV20083305             | MSL40                     |
|  2 | ICTV19740021    | Alphaentomopoxvirus         | ICTV19910168             | MSL40                     |
|  3 | ICTV19740021    | Betaentomopoxvirus          | ICTV19910169             | MSL40                     |
|  4 | ICTV19710040    | Poxviridae                  | ICTV19740004             | MSL40                     |
|  5 | ICTV19820133    | Alphacytorhabdovirus        | ICTV202419770            | MSL40                     |
|  6 | ICTV19820133    | Betacytorhabdovirus         | ICTV202419771            | MSL40                     |
|  7 | ICT